<a href="https://colab.research.google.com/github/boyoung21/ESAA/blob/main/1006_%EB%AC%B8%EC%84%9C_%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 군집화

비슷한 텍스트 구성의 문서를 군집화하는 것. 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있도록 해 주며, 비지도학습 기반으로 동작.

## opinion review 데이터 세트 이용한 문서 군집화 수행

In [32]:
import pandas as pd
import glob, os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 700)

path = r'/content/drive/MyDrive/OpinosisDataset1.0/topics'

# path로 지정한 디렉터리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합,
# 개별 파일들의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    # 절대 경로로 주어진 파일명을 가공. Linux에서 수행 시에는 아래 \\을 / 변경.
    # 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    filename = os.path.splitext(os.path.basename(file_))[0]

    # 파일명 리스트와 파일 내용 리스트에 파일명과 파일 내용을 추가.
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일 내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb.txt,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...
1,screen_netbook_1005ha.txt,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...
2,directions_garmin_nuvi_255W_gps.txt,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...
3,speed_garmin_nuvi_255W_gps.txt,Another feature on the 255w is a display of the posted speed limit on the road which you are currently on right above your current displayed speed .\n0 I found myself not even looking at my car speedometer as I could easily see my current speed and the speed limit of my route at a glance .\n1 ...
4,satellite_garmin_nuvi_255W_gps.txt,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ..."


해당 문서에 TF-IDF 형태로 픽처 벡터화 진행

TfidfVectorizer는 Lemmatization 같은 어근 변환을 직접 지원하진 않지만 tokenizer 인자에 커스텀 어근 변환 함수를 적용해 변환을 수행할 수 있음. 이를 위해 LemNormalize() 함수 생성.

In [36]:
import nltk

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer



# 필수 NLTK 데이터 다운로드 (필요시)

nltk.download('punkt')

nltk.download('punkt_tab')

nltk.download('wordnet')

def LemNormalize(text):

  lemmatizer = WordNetLemmatizer()

  return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words = 'english',
                             ngram_range = (1, 2), min_df = 0.05, max_df = 0.85)

# opinion_text 칼럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

K-평균 군집화를 통해 어떤 문서끼리 군집되는지 확인. 문서의 유형은 전자제품, 자동차, 호텔로 되어 있으며 먼저 5개의 Centroid 기반으로 어떻게 군집화되는지 확인. Kmeans를 수행한 후에 군집의 Label 값과 중심별로 할당된 데이터 세트의 좌표값 구함.

In [38]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state = 0
km_cluster = KMeans(n_clusters = 5, max_iter = 10000, random_state = 0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_cneters = km_cluster.cluster_centers_

In [39]:
# 군집이 잘 이루어졌는지 확인하기 위해 'cluster_label' 칼럼 추가

document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb.txt,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,1
1,screen_netbook_1005ha.txt,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,4
2,directions_garmin_nuvi_255W_gps.txt,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,4
3,speed_garmin_nuvi_255W_gps.txt,Another feature on the 255w is a display of the posted speed limit on the road which you are currently on right above your current displayed speed .\n0 I found myself not even looking at my car speedometer as I could easily see my current speed and the speed limit of my route at a glance .\n1 ...,4
4,satellite_garmin_nuvi_255W_gps.txt,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",4


In [40]:
# 군집화 결과 확인
# 0은 호텔 관련 리뷰

document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
38,location_bestwestern_hotel_sfo.txt,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",0
27,location_holiday_inn_london.txt,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",0
30,price_holiday_inn_london.txt,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0


In [41]:
# 1은 전자기기 및 전자기기의 주요 구성요소들인 배터리/키보드에 대한 리뷰들

document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
35,battery-life_amazon_kindle.txt,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ...",1
0,battery-life_ipod_nano_8gb.txt,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...,1
16,battery-life_netbook_1005ha.txt,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now...",1
6,features_windows7.txt,"I had to uninstall anti, virus and selected other programs, some of which did not have listings in the Programs and Features Control Panel section .\n0 This review briefly touches upon some of the key features and enhancements of Microsoft's latest OS .\n1 ...",1
42,performance_honda_accord_2008.txt,"Very happy with my 08 Accord, performance is quite adequate it has nice looks and is a great long, distance cruiser .\n0 6, 4, 3 eco engine has poor performance and gas mileage of 22 highway .\n1 Overall performance is good but comfort level is poor .\n2 ...",1
26,performance_netbook_1005ha.txt,"The Eee Super Hybrid Engine utility lets users overclock or underclock their Eee PC's to boost performance or provide better battery life depending on their immediate requirements .\n0 In Super Performance mode CPU, Z shows the bus speed to increase up to 169 .\n1 One...",1
14,size_asus_netbook_1005ha.txt,"A few other things I'd like to point out is that you must push the micro, sized right angle end of the ac adapter until it snaps in place or the battery may not charge .\n0 The full size right shift k...",1
8,sound_ipod_nano_8gb.txt,headphone jack i got a clear case for it and it i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1 ...,1
7,speed_windows7.txt,"Windows 7 is quite simply faster, more stable, boots faster, goes to sleep faster, comes back from sleep faster, manages your files better and on top of that it's beautiful to look at and easy to use .\n0 , faster about 20% to 30% faster at running applications than my Vista , seriously\n1 ...",1
5,video_ipod_nano_8gb.txt,"I bought the 8, gig Ipod Nano that has the built, in video camera .\n0 Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod .\n1 ...",1


In [42]:
# 2는 대부분 호텔

document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
50,bathroom_bestwestern_hotel_sfo.txt,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ...",2
33,food_holiday_inn_london.txt,The room was packed to capacity with queues at the food buffets .\n0 The over zealous staff cleared our unfinished drinks while we were collecting cooked food and movement around the room with plates was difficult in the crowded circumstances .\n1 ...,2
19,food_swissotel_chicago.txt,The food for our event was delicious .\n0 ...,2
48,free_bestwestern_hotel_sfo.txt,The wine reception is a great idea as it is nice to meet other travellers and great having access to the free Internet access in our room .\n0 They also have a computer available with free internet which is a nice bonus but I didn't find that out till the day before we left but was still able to get on there to check our flight to Vegas the next day .\n1 ...,2
45,parking_bestwestern_hotel_sfo.txt,Parking was expensive but I think this is common for San Fran .\n0 there is a fee for parking but well worth it seeing no where to park if you do have a car .\n1 ...,2
21,room_holiday_inn_london.txt,"We arrived at 23,30 hours and they could not recommend a restaurant so we decided to go to Tesco, with very limited choices but when you are hingry you do not careNext day they rang the bell at 8,00 hours to clean the room, not being very nice being waken up so earlyEvery day they gave u...",2
44,rooms_bestwestern_hotel_sfo.txt,"Great Location , Nice Rooms , H...",2
18,rooms_swissotel_chicago.txt,"The Swissotel is one of our favorite hotels in Chicago and the corner rooms have the most fantastic views in the city .\n0 The rooms look like they were just remodled and upgraded, there was an HD TV and a nice iHome docking station to put my iPod so I could set the alarm to wake up with my music instead of the radio .\n1 ...",2
39,service_bestwestern_hotel_sfo.txt,"Both of us having worked in tourism for over 14 years were very disappointed at the level of service provided by this gentleman .\n0 The service was good, very friendly staff and we loved the free wine reception each night .\n1 ...",2
20,service_holiday_inn_london.txt,"not customer, oriented hotelvery low service levelboor reception\n0 The room was quiet, clean, the bed and pillows were comfortable, and the serv...",2


In [43]:
# 도요타 및 혼다

document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
49,comfort_honda_accord_2008.txt,"Drivers seat not comfortable, the car itself compared to other models of similar class .\n0 ...",3
22,comfort_toyota_camry_2007.txt,"Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 Seats are fine, in fact of all the smaller sedans this is the most comfortable I found for the price as I am 6', 2 and 250# .\n1 Great gas mileage and comfortable on long trips ...",3
31,gas_mileage_toyota_camry_2007.txt,Ride seems comfortable and gas mileage fairly good averaging 26 city and 30 open road .\n0 ...,3
46,interior_honda_accord_2008.txt,I love the new body style and the interior is a simple pleasure except for the center dash .\n0 ...,3
24,interior_toyota_camry_2007.txt,"First of all, the interior has way too many cheap plastic parts like the cheap plastic center piece that houses the clock .\n0 3 blown struts at 30,000 miles, interior trim coming loose and rattling squeaking, stains on paint, and bug splats taking paint off, premature uneven brake wear, on 3rd windsh...",3
41,mileage_honda_accord_2008.txt,"It's quiet, get good gas mileage and looks clean inside and out .\n0 The mileage is great, and I've had to get used to stopping less for gas .\n1 Thought gas ...",3
47,quality_toyota_camry_2007.txt,I previously owned a Toyota 4Runner which had incredible build quality and reliability .\n0 I bought the Camry because of Toyota reliability and qua...,3
40,seats_honda_accord_2008.txt,"Front seats are very uncomfortable .\n0 No memory seats, no trip computer, can only display outside temp with trip odometer .\n1 ...",3
43,transmission_toyota_camry_2007.txt,"After slowing down, transmission has to be kicked to speed up .\n0 ...",3


군집 개수가 데이터에 비해 약간 많게 설정되어 있어서 더 세분화되어 군집화된 경향 있음. 0과 2가 비슷한 호텔에 대한 리뷰인 것처럼. 군집 개수를 3개로 줄여서 다시 결과 확인.

In [44]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

# 소속 클러스터를 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
2,directions_garmin_nuvi_255W_gps.txt,You also get upscale features like spoken directions including street names and programmable POIs .\n0 I used to hesitate to go out of my directions but no...,0
13,display_garmin_nuvi_255W_gps.txt,"3 quot widescreen display was a bonus .\n0 This made for smoother graphics on the 255w of the vehicle moving along displayed roads, where the 750's display was more of a jerky movement .\n1 ...",0
27,location_holiday_inn_london.txt,"Great location for tube and we crammed in a fair amount of sightseeing in a short time .\n0 All in all, a normal chain hotel on a nice lo...",0
30,price_holiday_inn_london.txt,"All in all, a normal chain hotel on a nice location , I will be back if I do not find anthing closer to Picadilly for a better price .\n0 ...",0
38,location_bestwestern_hotel_sfo.txt,"Good Value good location , ideal choice .\n0 Great Location , Nice Rooms , Helpless Concierge\n1 ...",0
4,satellite_garmin_nuvi_255W_gps.txt,"It's fast to acquire satellites .\n0 If you've ever had a Brand X GPS take you on some strange route that adds 20 minutes to your trip, has you turn the wrong way down a one way road, tell you to turn AFTER you've passed the street, frequently loses the satellite signal, or has old maps missing streets, you know how important this stuff is .\n1 ...",1
1,screen_netbook_1005ha.txt,Keep in mind that once you get in a room full of light or step outdoors screen reflections could become annoying .\n0 I've used mine outsi...,1
5,video_ipod_nano_8gb.txt,"I bought the 8, gig Ipod Nano that has the built, in video camera .\n0 Itunes has an on, line store, where you may purchase and download music and videos which will install onto the ipod .\n1 ...",1
7,speed_windows7.txt,"Windows 7 is quite simply faster, more stable, boots faster, goes to sleep faster, comes back from sleep faster, manages your files better and on top of that it's beautiful to look at and easy to use .\n0 , faster about 20% to 30% faster at running applications than my Vista , seriously\n1 ...",1
8,sound_ipod_nano_8gb.txt,headphone jack i got a clear case for it and it i got a clear case for it and it like prvents me from being able to put the jack all the way in so the sound can b messsed up or i can get it in there and its playing well them go to move or something and it slides out .\n0 Picture and sound quality are excellent for this typ of devic .\n1 ...,1


# 군집별 핵심 단어 추출하기

각 군집에 속한 문서는 핵심 단어를 주축으로 군집화되어 있음. 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지는 clusters_centers_ 속성으로 제공. 이는 배열 값으로 제공되며, 행은 개별 군집을, 열은 개별 피처를 의미함. 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값. 예를 들어 cluster_centers[0,1]은 0번 군집에서 두 번째 피처의 위치 값.

In [45]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape: ', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape:  (3, 6155)
[[0.         0.         0.00077394 ... 0.         0.         0.        ]
 [0.00123579 0.00090868 0.         ... 0.         0.         0.        ]
 [0.         0.00050408 0.00081489 ... 0.00216051 0.00171142 0.00171142]]


In [50]:
# ndarray의 argsort()[:,::-1]를 이용해 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값 반환
# 피처 이름을 출력하기 위해 인덱스 값을 반환하는 것

# 군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
  cluster_details = {}

  # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값을 반환
  # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
  centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num] = {}
    cluster_details[cluster_num]['cluster'] = cluster_num

    # cluster_centers_.argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features = [feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값을 구함.
    top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features'] = top_features
    cluster_details[cluster_num]['top_features_value'] = top_feature_values
    filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
    filenames = filenames.values.tolist()

    cluster_details[cluster_num]['filenames'] = filenames

  return cluster_details

해당 함수를 시행하면 dictionary를 원소로 가지는 리스트인 cluster_details를 반환

호출 인자: KMeans 군집화 객체, 파일명 추출을 위한 document_df, 핵심 단어 추출을 위한 피처명 리스트, 전체 군집 개수, 핵심 단어 추출 개수

이 cluster_details에는 개별 군집번호, 핵심 단어, 핵심단어 중심 위치 상댓값, 파일명 속성 값 정보가 있음. 이를 더 좋게 표현하기 위해 print_cluster_details() 함수 생성.

In [51]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features: ', cluster_detail['top_features'])
    print('Reviews 파일명: ', cluster_detail['filenames'][:7])
    print("==============================================")

In [52]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                      feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features:  ['location', 'hotel', 'direction', 'price', 'display', 'great location', 'location .', 'location ,', 'wharf', 'tube']
Reviews 파일명:  ['directions_garmin_nuvi_255W_gps.txt', 'display_garmin_nuvi_255W_gps.txt', 'location_holiday_inn_london.txt', 'price_holiday_inn_london.txt', 'location_bestwestern_hotel_sfo.txt']
####### Cluster 1
Top features:  ['screen', 'battery', 'interior', 'seat', 'performance', 'mileage', 'keyboard', 'battery life', 'life', 'kindle']
Reviews 파일명:  ['battery-life_ipod_nano_8gb.txt', 'screen_netbook_1005ha.txt', 'speed_garmin_nuvi_255W_gps.txt', 'satellite_garmin_nuvi_255W_gps.txt', 'video_ipod_nano_8gb.txt', 'features_windows7.txt', 'speed_windows7.txt']
####### Cluster 2
Top features:  ['room', 'service', 'staff', 'hotel', 'food', 'bathroom', 'clean', 'parking', 'room service', 'free']
Reviews 파일명:  ['rooms_swissotel_chicago.txt', 'food_swissotel_chicago.txt', 'service_holiday_inn_london.txt', 'room_holiday_inn_london.txt', 'servic